In [ ]:
from selenium import webdriver
from selenium.webdriver.edge import service as fs
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
import pandas as pd

'''
Harvey Normanのサイトから機種名とレビュー内容を取得
'''
def get_review(url):
    #=== ページ設定 ===
    css_productname = '#overview > div.col-xs-12.l_mgn-b-sm > small:nth-child(2)'
    css_reviewtabbutton = '#tab-product-reviews'
    css_number = '#BVRRContainer > div > div > div > div > div.bv-header > div.bv-control-bar > div.bv-control-bar-count > div > span'
    class_review = 'bv-content-item-author-profile-offset-on'
    class_body = 'bv-content-summary-body-text'
    class_title = 'bv-content-title-container'
    css_nextbutton = '#BVRRContainer > div > div > div > div > div.bv-content-pagination > div > ul > li.bv-content-pagination-buttons-item.bv-content-pagination-buttons-item-next > a'
    #================
    
    # URLを開く
    edge_service = fs.Service(executable_path=r'~./edgedriver_mac64/msedgedriver')
    driver = webdriver.Edge(service=edge_service)
    driver.get(url)
    # driver.set_window_size('1500','1000')

    # 機種名取得
    model_name = driver.find_element(By.CSS_SELECTOR, css_productname).text
    print('------ ' + model_name + ' ------')

    # レビュータブを開く
    driver.find_element(By.CSS_SELECTOR,css_reviewtabbutton).click()
    driver.implicitly_wait(6)

    # ページ数，レビュー数の取得
    # - "1-10 of 100 Reviews" を取得し，ページ数やレビュー数の確認をできるようにする
    npage_review = driver.find_element(By.CSS_SELECTOR, css_number).text
    nreview = int(npage_review.split()[2])
    # nreview = int(tmp[2])

    # 本文とタイトルを取得
    print('start >>>', end=' ')
    reviews = []
    titles = []
    print('page #', end=' ')
    for ipage in range(50):
        print(str(ipage+1), end=',')

        # ページ内の複数レビューカードを取得
        elms_review = driver.find_elements(By.CLASS_NAME, class_review)
        # レビューカードから本文，タイトル取得
        for elm in elms_review:
            try:
                elm_body = elm.find_element(By.CLASS_NAME, class_body)
                elm_title = elm.find_element(By.CLASS_NAME, class_title)
                reviews.append(elm_body.text)
                titles.append(elm_title.text)
            except NoSuchElementException:
                print('skip', end=',')

        # 次のページへ遷移
        try:
            elm = driver.find_element(by=By.CSS_SELECTOR, value=css_nextbutton)
        except NoSuchElementException:
            print(' >>> end')
            break
        elm.click()
        time.sleep(1)

    # 確認用
    print('取得ページ数: ' + str(ipage+1))
    print('取得レビュー数: ' + str(len(reviews))+ '/' + str(nreview))    
    print('-----------------------')

    reviews = [s.replace('\n','.') for s in reviews]
    titles = [s.replace('\n','.') for s in titles]
    df = pd.DataFrame(list(zip(titles,reviews)), columns = ['title','review'])
    return model_name, df



In [ ]:
url_list = [
            # 'https://www.harveynorman.com.au/westinghouse-565l-stainless-steel-french-door-fridge-with-water-dispenser.html',
            # 'https://www.harveynorman.com.au/lg-637l-french-door-fridge-with-instaview-door-in-door.html',
            # 'https://www.harveynorman.com.au/electrolux-609l-dark-stainless-steel-french-door-fridge.html',
            # 'https://www.harveynorman.com.au/samsung-498l-french-door-refrigeration-silver-layered-steel.html',
            # 'https://www.harveynorman.com.au/lg-508l-slim-french-door-fridge-with-ice-and-water-dispenser-matte-black.html',
            # 'https://www.harveynorman.com.au/samsung-488l-french-door-refrigeration-black-layered-steel.html',
            # 'https://www.harveynorman.com.au/westinghouse-609l-french-door-fridge-with-ice-and-water-dispenser-dark-stainless-steel.html',
            # 'https://www.harveynorman.com.au/samsung-488l-french-door-refrigeration-silver-layered-steel.html',
            # 'https://www.harveynorman.com.au/westinghouse-609l-stainless-steel-french-door-fridge-with-ice-water-dispenser.html',
            # 'https://www.harveynorman.com.au/lg-637l-french-door-fridge-with-door-in-door.html',
            # 'https://www.harveynorman.com.au/lg-637l-french-door-fridge-with-slim-in-door-icemaker-matte-black.html',
            # 'https://www.harveynorman.com.au/lg-637l-french-door-fridge-with-slim-in-door-icemaker.html',
            # 'https://www.harveynorman.com.au/lg-852l-french-door-fridge-with-instaview-door-in-door-matte-black.html',
            # 'https://www.harveynorman.com.au/lg-508l-slim-french-door-fridge-with-instaview-door-in-door.html',
            # 'https://www.harveynorman.com.au/westinghouse-491l-bar-handle-french-door-fridge-dark-stainless-steel.html',
            # 'https://www.harveynorman.com.au/westinghouse-541l-stainless-steel-french-door-fridge-with-water-dispenser.html',
            # 'https://www.harveynorman.com.au/lg-508l-slim-french-door-fridge-with-non-plumbed-ice-and-water-dispenser-matte-black.html',
            ]

In [ ]:
def save_to_file(tpl):
    model_name = tpl[0]
    df = tpl[1]
    df.to_csv('./reviews/reviews_' + model_name + '.csv', index=False, encoding='utf_8_sig')

for url in url_list:
    save_to_file(get_review(url))